In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output, display

# Define the simulation environment for a three-wheeled omniwheel robot
class Simulation:
    def __init__(self, robot):
        self.robot = robot
        self.dt = 0.1
        self.time = 0
        self.history = []
        self.figure = plt.figure()
        self.ax = self.figure.add_subplot(111)
        self.run = False

    
    def step(self, v, w, z, window_size = [20, 20]):
        """
        Get the wheel rotation speeds for each wheel and pass
        them to the robot to update the pose.
        """
        self.robot.step(v, w, z, self.dt)
        self.time += self.dt
        self.history.append(self.robot.get_pose())
        if not self.run:
            self.plot(window_size)

    def run(self, v, w, z, duration):
        """
        Run the simulation for a given duration with the given
        rotation speeds for each wheel.
        """
        self.run = True
        for _ in range(int(duration / self.dt)):
            self.step(v, w, z)
            self.plot()
    
    def plot(self, window_size):
        axis_lower_lim = window_size[0]
        axis_upper_lim = window_size[1]
        self.ax.cla()
        history = np.array(self.history)
        self.ax.plot(history[:, 0], history[:, 1], linewidth=2, color='red')
        self.ax.set_xlim(axis_lower_lim, axis_upper_lim)
        self.ax.set_ylim(axis_lower_lim, axis_upper_lim)
        #self.ax.axis('equal')
        self.ax.set_xlabel('x')
        self.ax.set_ylabel('y')
        self.ax.set_title('Robot Trajectory')
        self.ax.legend(['Trajectory'])
        self.plot_body()
        
        clear_output(wait=True)
        display(self.figure)
        plt.draw()
        plt.pause(0.1)

    def plot_body(self):
        # Define the vertices of an equilateral triangle
        side_length = self.robot.wheel_distance
        height = np.sqrt(3) / 2 * side_length  # Height of the equilateral triangle
        # Vertices of the equilateral triangle
        centre = np.array([0, 0])
        vertices = np.array([
            [-side_length/2, -height/3],  # First vertex at origin
            [side_length/2, -height/3],  # Second vertex
            [0, 2*height/3]  # Third vertex
        ])

        # Calculate transformed vertices
        theta = self.robot.theta
        R = np.array([
            [np.cos(theta), -np.sin(theta)],
            [np.sin(theta), np.cos(theta)]
        ])
        transformed_centre = 10 * centre @ R.T + np.array([[self.robot.x, self.robot.y]])
        transformed_vertices = 10 * vertices @ R.T + np.array([[self.robot.x, self.robot.y]])

        # Close the triangle by adding the first vertex at the end
        triangle = np.vstack([transformed_vertices, transformed_vertices[0]])
        self.ax.plot(transformed_centre[0,0], transformed_centre[0,1], 'ro')
        self.ax.plot(triangle[:, 0], triangle[:, 1], 'bo-', linewidth=2) 
        self.ax.fill(triangle[:, 0], triangle[:, 1], 'skyblue', alpha=0.5)
        self.ax.grid(True)

In [ ]:
import numpy as np

class Robot:
    def __init__(self, x=0, y=0, theta=0):
        self.x = x
        self.y = y
        self.theta = theta
        self.wheel_radius = 0.1         # Radius of the wheels in meters
        self.wheel_distance = 0.2       # Distance of wheel connectons to the centre of triangle in meters
        self.wheel_speeds = [0, 0, 0]
        self.H_matrix = (1/self.wheel_radius)*np.array([
            [-self.wheel_distance, 1, 0],
            [self.wheel_distance, -0.5, -np.sqrt(3)/2],
            [self.wheel_distance, -0.5, np.sqrt(3)/2]])
        self.H_inv = np.linalg.inv(self.H_matrix)
    
    def step(self, v, w, z, dt):
        """
        Update the robot's pose based on the wheel speeds.
        """
        self.theta_dot, self.x_dot, self.y_dot = np.dot(self.H_inv, [v, w, z])
        
        self.theta += self.theta_dot * dt
        self.x += self.x_dot * dt
        self.y += self.y_dot * dt
    
    def find_wheel_speeds(self, x_dot, y_dot, theta_dot):
        """
        Find the wheel speeds based on the robot's pose.
        """
        v, w, z = np.dot(self.H_matrix, [theta_dot, x_dot, y_dot])
        return v, w, z
    
    def get_pose(self):
        return self.x, self.y, self.theta

A stright line with a slope of 60

In [ ]:
import numpy as np

def sixty_degree_line_path(robot, time):
    x_dot = 1
    y_dot = np.sqrt(3)
    v, w, z = robot.find_wheel_speeds(x_dot, y_dot, 0)
    return v, w, z

def main():
    # Create a robot object
    robot = Robot()
    
    # Create a simulation object
    sim = Simulation(robot)
    
    # Run the simulation
    dt = 0.1
    for _ in range(int(30/dt)):
        v, w, z = sixty_degree_line_path(robot, sim.time)
        sim.step(v, w, z, window_size=[-5, 55])
    
    # Plot the trajectory
    sim.plot()

if __name__ == '__main__':
    main()

a 2 meters-diameter circle

In [ ]:
import numpy as np

def circle_path(robot, time):
    x_dot = -np.sin(time)
    y_dot = np.cos(time)
    v, w, z = robot.find_wheel_speeds(x_dot, y_dot, 0)
    return v, w, z

def main():
    # Create a robot object
    robot = Robot()
    
    # Create a simulation object
    sim = Simulation(robot)
    
    # Run the simulation
    dt = 0.1

    for _ in range(int(30/dt)):
        v, w, z = circle_path(robot, sim.time)
        sim.step(v, w, z, window_size=[-3,3])
    
    # Plot the trajectory
    sim.plot()

if __name__ == '__main__':
    main()

An expanding spiral of any size (the chosen size is shown in the report)

In [ ]:
import numpy as np

def expanding_spiral_path(robot, time):
    x_dot = np.cos(time) - time * np.sin(time)
    y_dot = np.sin(time) + time * np.cos(time)
    v, w, z = robot.find_wheel_speeds(x_dot, y_dot, 0)
    return v, w, z

def main():
    # Create a robot object
    robot = Robot()
    
    # Create a simulation object
    sim = Simulation(robot)
    
    # Run the simulation
    dt = 0.1

    for _ in range(int(30/dt)):
        v, w, z = expanding_spiral_path(robot, sim.time)
        sim.step(v, w, z, window_size=[-30,30])
    
    # Plot the trajectory
    sim.plot()

if __name__ == '__main__':
    main()